# Part 1 - Ciphers
## Eyal Rechef - 213099468, Alon Abudraham - 324231992
### Introduction
We have three files containing ciphertexts, which all represent the same plaintext.
Each file is using one of the following ciphers:
* Vigenère cipher
* Hill cipher
* An advanced encryption scheme

Each scheme uses a **block size of 5**.

Let us write some utility functions which will help us handle the files:

In [284]:
FILE_NAME = "file{index}.txt"
DECRYPTED_FILE_NAME = "file{index}.dec.txt"
FILE_NUM = 3

CIPHER_BLOCK_SIZE = 5
ALPHABET_SIZE = 26
ALPHABET_BASE = "A"


def letter_to_number(letter: str) -> int:
    """Maps A-Z letters to numbers 0-25"""
    assert len(letter) == 1, f"Not a single character"
    assert letter.isupper(), f"Not in A-Z"
    return ord(letter) - ord(ALPHABET_BASE)


def number_to_letter(number: int) -> str:
    assert 0 <= number <= ALPHABET_SIZE, f"Number must be between 0 and {ALPHABET_SIZE}"
    return chr(number + ord(ALPHABET_BASE))


def block_to_string(block: list[int]) -> str:
    return "".join(number_to_letter(number) for number in block)


def string_to_num_block(string: str) -> list[int]:
    return [letter_to_number(char) for char in string]


def split_list_to_block(
    lst: list[int], block_size: int = CIPHER_BLOCK_SIZE
) -> list[list[int]]:
    """Splits a list into blocks of block_size"""
    return [lst[i : i + block_size] for i in range(0, len(lst), block_size)]


def get_string_as_blocks(
    text: str, block_size: int = CIPHER_BLOCK_SIZE
) -> list[list[int]]:
    return split_list_to_block(string_to_num_block(text), block_size)


def get_file_as_string(index: int) -> str:
    with open(FILE_NAME.format_map({"index": index}), "r") as fd:
        return fd.read()


def get_file_blocks(index: int) -> list[list[int]]:
    return get_string_as_blocks(get_file_as_string(index))


def get_as_str(text: list[list[int]]) -> str:
    """Gives the string behind our format (list of blocks of ints)"""
    return block_to_string([item for sublist in text for item in sublist])


def write_plaintext(index: int, plaintext: list[list[int]]):
    """Writes result to plaintext file"""
    with open(DECRYPTED_FILE_NAME.format_map({"index": index}), "w") as fd:
        fd.write(get_as_str(plaintext))



### 1. Recover the plaintext
We need a method to determine the original plaintext. Right now our attacks are COA, we can perform a frequent analysis.

We'll assume the key is 5 letters long (like the block), and go over each one of the files.

We'll take `E` (value of `4`) as the most frequent letter in the alphabet.

For each of the 5 indices of the blocks, we'll expect a different value for the most frequent letter, that way we will get the key.

For each plaintext character in a block, the relation to the ciphertext will be:

$$ c = (p + k) \mod 26$$

In [285]:
import numpy as np
from collections import Counter

MOST_FREQUENT_CHAR = 4  # E corresponding


def vigenere_decrypt(cipher_text: list[list[int]], key: list[int]) -> list[list[int]]:
    plaintext = []
    for block in cipher_text:
        plaintext.append(np.subtract(block, key) % ALPHABET_SIZE)
    return plaintext


def vigenere_frequent_analysis_attack(file_index: int) -> tuple[str, str]:
    ciphertext = get_file_blocks(i + 1)

    # Instead of list as blocks - list as list of chars in the 1st index, 2nd index etc.
    ciphertext_by_index = [list(item) for item in list(zip(*ciphertext))]

    key = []
    for list_of_index in ciphertext_by_index:
        most_common_ciphertext = Counter(list_of_index).most_common(1)[0][0]
        # The current item of the key is relative to E
        key.append((most_common_ciphertext - MOST_FREQUENT_CHAR) % ALPHABET_SIZE)

    return get_as_str(vigenere_decrypt(ciphertext, key)), key


for i in range(FILE_NUM):
    plaintext, key = vigenere_frequent_analysis_attack(i)
    print("-" * 50)
    print(f"Key found for file #{i+1} is '{key}'")
    print(plaintext[:100])

--------------------------------------------------
Key found for file #1 is '[0, 9, 9, 6, 13]'
EYMPEXIOOBWVNBQIYJWDTKPNSSFQTYZMTKCGINVKYVYDCAXKEWVCGYFHGHDBMUKUKMZLBXYAAVTLYGHFLMUPKHRWZIOLZCZBNUYJ
--------------------------------------------------
Key found for file #2 is '[5, 11, 9, 1, 23]'
UIRXLYKFAIBCRGFZLVHMPUBLOPWJWHJRPSZJIQJTOYJBBVKVEEETWQHVHHDPKKSDCLIVOLFUKTTRBEVATPLDARTHXDJMVDEXIDQF
--------------------------------------------------
Key found for file #3 is '[17, 8, 6, 7, 19]'
INTHEBEGINNINGGODCREATEDTHEHEAVENANDTHEEARTHANDTHEEARTHWASWITHOUTFORMANDVOIDANDDARKNESSWASUPONTHEFAC


Looking at the result, it seems our attack worked!

The result of `file3.txt` seems like the content of the Genesis book, while the others seem like noise. 

The key which got us this result was `"RIGHT"`!

### 2 Identify and derive the key from the remaining cipher

We've established that `file3.txt` corresponds to Vigenère cipher.

We need to find which of the remaining corresponds to Hill cipher and extract its key.

We have more than 5 blocks of plaintext and ciphertext. 

We can construct two 5x5 matrices, constructed of plaintext and ciphertext blocks $C, P \in \{0, \dots, 25\}^{5\times 5}$, 
from this we can extract the key matrix $K$:

$$K = C\cdot P^{-1} \mod 26$$

Of course $P$ has to be invertible, but we have a big enough sample of plaintext ciphertext couples to generate such matrix.

In order to calculate the inverse, we'll write python functions for adjugate and modular inverse.

In [286]:
VIGENERE_FILE_INDEX = 3
CORRECT_PLAINTEXT = vigenere_frequent_analysis_attack(VIGENERE_FILE_INDEX)[0]
PLAINTEXT_AS_BLOCKS = get_string_as_blocks(CORRECT_PLAINTEXT)
NUM_OF_FILES_NOT_CLASSIFIED = 2


def adjugate(mat: np.array) -> np.array:
    n = mat.shape[0]
    adj = np.zeros_like(mat)

    for i in range(n):
        for j in range(n):
            minor = np.delete(np.delete(mat, i, axis=0), j, axis=1)
            cofactor = ((-1) ** (i + j)) * int(round(np.linalg.det(minor)))
            adj[j, i] = cofactor

    return adj


def modular_matrix_inversion(mat: np.array, modulo: int) -> np.array:
    adj = adjugate(mat) % modulo
    det_inv = pow(int(round(np.linalg.det(mat))) % modulo, -1, mod=modulo)
    return (det_inv * adj) % modulo


def get_matrix_from_blocks_vectors(blocks: list[list[int]], offset: int) -> np.array:
    """
    Creates a matrix of vectors (vector representation of blocks) from first
    CIPHER_BLOCK_SIZE blocks in a message
    """
    return np.array(blocks[offset : offset + CIPHER_BLOCK_SIZE]).T


def hill_cipher_key_extract(file_index: int, offset: int):
    ciphertext_as_blocks = get_file_blocks(file_index)
    # Taking first 5 blocks for each matrix
    p_mat = get_matrix_from_blocks_vectors(PLAINTEXT_AS_BLOCKS, offset)
    c_mat = get_matrix_from_blocks_vectors(ciphertext_as_blocks, offset)

    return c_mat @ modular_matrix_inversion(p_mat, ALPHABET_SIZE) % ALPHABET_SIZE


def hill_encrypt(
    plaintext_as_blocks: list[list[int]], key: np.array
) -> list[list[int]]:
    return [
        (np.array(key @ np.array(block).T) % ALPHABET_SIZE).tolist()
        for block in plaintext_as_blocks
    ]


def hill_decrypt(
    ciphertext_as_blocks: list[list[int]], key: np.array
) -> list[list[int]]:
    key_inv = modular_matrix_inversion(key, ALPHABET_SIZE)
    return [
        (np.array(key_inv @ np.array(block).T) % ALPHABET_SIZE).tolist()
        for block in ciphertext_as_blocks
    ]


for i in range(NUM_OF_FILES_NOT_CLASSIFIED):
    offset = 0
    while 1:
        try:
            key = hill_cipher_key_extract(i + 1, offset)
            # Try decrypt using key
            break
        except ValueError:
            # Picked few vectors which cannot be inverted - shift forward
            offset += 1

    print("-" * 50)
    print(f"Extracted key for file {i+1} is \n{key}")
    try:
        plaintext = hill_decrypt(get_file_blocks(i + 1), key)
        if get_as_str(plaintext) == CORRECT_PLAINTEXT:
            print(f"For file {i+1} - Hill decryption with found key worked!")
    except ValueError:
        pass

--------------------------------------------------
Extracted key for file 1 is 
[[22 24 10 22  7]
 [ 8 25 12 15 19]
 [11 11  4  8 21]
 [10 10  3  6 25]
 [15 16  0 19 10]]
--------------------------------------------------
Extracted key for file 2 is 
[[ 3  4  1 20  6]
 [ 6  8 13  6  5]
 [14 17  4 21  4]
 [17 13  4 21  4]
 [17  4 13  3 18]]
For file 2 - Hill decryption with found key worked!


### 3. Report both keys and match each file to its cipher

So now we can say:

- `file1.txt` is the one with the advanced encryption scheme.
- `file2.txt` is encrypted with Hill cipher, and a key of $\begin{pmatrix}
3 & 4 & 1 & 20 & 6\\
6 & 8 & 13 & 6 & 5\\
14 & 17 & 4 & 21 & 4\\
17 & 13 & 4 & 21 & 4\\
17 & 4 & 13 & 3 & 18
\end{pmatrix}$
- `file3.txt` is encrypted with Vigenère cipher, and a key of `"RIGHT"`


### 

### 4. Decrypt the assistant's secret message

Now we have a hold on a lot of ciphertext and plaintext block couples. 

We can create a dictionary based `{known_ciphertext_block: known_plaintext_block}` and hope we have all the words.

Then we can go over the secret message and use its blocks as keys to get the plaintext.

In [287]:
ADVANCED_CIPHER_FILE = 1
SECRET_MESSAGE_FILE = "secret_message.txt"

attack_dict = dict()
ciphertext = get_file_as_string(ADVANCED_CIPHER_FILE)

# Generate attack dictionary
for i in range(0, len(ciphertext), CIPHER_BLOCK_SIZE):
    attack_dict[ciphertext[i : i + CIPHER_BLOCK_SIZE]] = CORRECT_PLAINTEXT[
        i : i + CIPHER_BLOCK_SIZE
    ]

with open(SECRET_MESSAGE_FILE) as fd:
    secret_message = fd.read()

decrypted_secret_message = ""
for i in range(0, len(secret_message), CIPHER_BLOCK_SIZE):
    decrypted_secret_message += attack_dict[secret_message[i : i + CIPHER_BLOCK_SIZE]]

print(f"Secret message was {decrypted_secret_message}")

Secret message was ITWASTHEENEMYBROTHER


And we indeed found the secret message - `ITWASTHEENEMYBROTHER`